In [ ]:
!pip install spacy_transformers
!pip install -U spacy
!pip install spacy==3.6.0

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [ ]:
spacy.__version__

'3.6.0'

In [ ]:
!nvidia-smi

Wed Nov 22 17:14:34 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    12W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!git clone https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git

Cloning into 'CV-Parsing-using-Spacy-3'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 82 (delta 14), reused 74 (delta 14), pack-reused 6
Receiving objects: 100% (82/82), 5.62 MiB | 9.49 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [ ]:
cv_data = json.load(open('/content/CV-Parsing-using-Spacy-3/data/training/train_data.json','r'))

In [ ]:
len(cv_data)

200

In [ ]:
!python -m spacy init fill-config /content/CV-Parsing-using-Spacy-3/data/training/base_config.cfg /content/CV-Parsing-using-Spacy-3/data/training/config.cfg
!python -m spacy download en_core_web_sm

2023-11-22 18:11:52.554040: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-22 18:11:52.554091: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-22 18:11:52.554133: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-22 18:11:53.679452: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
/content/CV-Parsing-using-Spacy-3/data/training/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./d

In [ ]:
#cv_data

In [ ]:
def get_spacy_doc(file, data):
    nlp = spacy.load("en_core_web_sm")
    db = DocBin()

    for text, annot in tqdm(data):
        doc = nlp(text)
        entities = annot['entities']

        ents = []
        entity_indices = set()

        for start, end, label in entities:
            skip_entity = False
            for idx in range(start, end):
                if idx in entity_indices:
                    skip_entity = True
                    break

            if skip_entity:
                continue

            entity_indices.update(range(start, end))

            try:
                span = doc.char_span(start, end, label=label, alignment_mode='strict')
            except:
                continue

            if span is None:
                err_data = f"{[start, end]}    {text}\n"
                file.write(err_data)
            else:
                ents.append(span)

        try:
            doc.ents = ents
            db.add(doc)
        except:
            pass

    return db






In [ ]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(cv_data,test_size=0.3)

In [ ]:
len(train), len(test)

(140, 60)

In [ ]:
file = open('error.txt','w')

db = get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('test_data.spacy')


file.close()

100%|██████████| 60/60 [00:09<00:00,  6.46it/s]


In [ ]:
len(db.tokens)


In [ ]:
!python -m spacy train /content/CV-Parsing-using-Spacy-3/data/training/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

2023-11-22 18:48:44.749320: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-22 18:48:44.749388: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-22 18:48:44.749425: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-22 18:48:46.480586: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-11-22 18:48:51,152] [INFO] Set up nlp object from config
[2023-11-22 18:48:51,169] [INFO] Pipeline: [

### MODEL TESTING


In [ ]:
nlp = spacy.load('/content/output/model-best')

In [ ]:
doc = nlp('my name is Laxmi Kant Tiwari. I worked at Microsoft.I have 10 years expereince')
for ent in doc.ents:
  print(ent.text,"   ->>>>>", ent.label_)

Laxmi Kant    ->>>>> Name
Microsoft    ->>>>> Companies worked at
10 years    ->>>>> Years of Experience


In [ ]:
!pip install PyMuPDF

In [ ]:
import sys, fitz

In [ ]:
fname='/content/CV-Parsing-using-Spacy-3/data/test/Alice Clark CV.pdf'
doc= fitz.open(fname)

In [ ]:
text = " "
for page in doc:
  text = text + str(page.get_text())


In [ ]:
text = text.strip()

In [ ]:
'  '.join(text.split())

'Alice  Clark  AI  /  Machine  Learning  Delhi,  India  Email  me  on  Indeed  •  20+  years  of  experience  in  data  handling,  design,  and  development  •  Data  Warehouse:  Data  analysis,  star/snow  flake  scema  data  modelling  and  design  specific  to  data  warehousing  and  business  intelligence  •  Database:  Experience  in  database  designing,  scalability,  back-up  and  recovery,  writing  and  optimizing  SQL  code  and  Stored  Procedures,  creating  functions,  views,  triggers  and  indexes.  Cloud  platform:  Worked  on  Microsoft  Azure  cloud  services  like  Document  DB,  SQL  Azure,  Stream  Analytics,  Event  hub,  Power  BI,  Web  Job,  Web  App,  Power  BI,  Azure  data  lake  analytics(U-SQL)  Willing  to  relocate  anywhere  WORK  EXPERIENCE  Software  Engineer  Microsoft  –  Bangalore,  Karnataka  January  2000  to  Present  1.  Microsoft  Rewards  Live  dashboards:  Description:  -  Microsoft  rewards  is  loyalty  program  that  rewards  Users  for

In [ ]:
doc = nlp(text)
for ent in doc.ents:
  print(ent.text,"   ->>>>>", ent.label_)

Alice Clark    ->>>>> Name
AI / Machine Learning    ->>>>> Designation
Delhi    ->>>>> Location
20+    ->>>>> Years of Experience
Software Engineer    ->>>>> Designation
Indian Institute of Technology    ->>>>> College Name
2001    ->>>>> Graduation Year
